## Theory

**0.** In the lecture we count Frobenious norm for Kruskal tensor $||\hat{Y}||$

Write up norm for the difference of two Kruskal tensors $||\hat{Y} - \hat{Y}||^2$ in the similar way, if $\hat{Y} = [|U V W|]$, $\hat{X} = [|A B C|]$

**Solution**
 
$||\hat{Y} - \hat{X}||^2 = ||\hat{Y}||^2 + ||\hat{X}||^2  - 2<\hat{Y}, \hat{X}>$

$||\hat{Y}||^2 = \hat{1}^T((\hat{U^T} \hat{U} \ast \hat{V^T} \hat{V} \ast \hat{W^T} \hat{W}))\hat{1}$ 

$||\hat{X}||^2 = \hat{1}^T((\hat{A^T} \hat{A} \ast \hat{B^T} \hat{B} \ast \hat{C^T} \hat{C}))\hat{1}$ 

$2<\hat{Y}, \hat{X}> =  \hat{1}^T((\hat{A^T} \hat{U} \ast \hat{B^T} \hat{V} \ast \hat{C^T} \hat{W}))\hat{1}$

**1.**
Given a Kruskal tensor $\hat{X} = [|U^{(0)};U^{(1)};U^{(2)}|]$ where the factor matrices $U^{(0)}$, $U^{(1)}$ and $U^{(2)}$. Write update rules of the ALS algorithm which finds a low-rank approximation using unfolding of initial tensor $\hat{X}$ to corrsponding mode and factor matrices, whithout using pseudo-inverse matrix.

**Solution**

1. In the lecture slides using LLS we take:
$U(0) = \hat{X_{[0]}}[ (U^{(1)} \odot U^{(2)})^T]^{\dagger}$
2. use property of Khartri-rao: $U(0) = \hat{X_{[0]}} (U^{(1)} \odot U^{(2)}) [(U^{(1)^T}(U^{(1)}) * (U^{(2)^T}(U^{(2)})] $

equations for another factors is the similar 

# Practice: ALS algorithm

Using formula from **1.** write up an als algoritm. Create some tensor and decompose it using obtained functionality

In [31]:
def unfold(tensor, mode):
    
    return np.reshape(np.moveaxis(tensor, mode, 0),
                      (tensor.shape[mode], -1))


def fold(unfolded_tensor, mode, shape):
    
    full_shape = list(shape)
    mode_dim = full_shape.pop(mode)
    full_shape.insert(0, mode_dim)
    return np.moveaxis(np.reshape(
          unfolded_tensor, full_shape), 0, mode)

In [32]:
def khatri_rao(matrix1, matrix2):
    n_columns = matrix1.shape[1]
    result = np.einsum('ik,jk->ijk', matrix1, matrix2)
    return result.reshape((-1, n_columns))

In [33]:
def factors_to_tensor(A, B, C):
    full_shape = (A.shape[0], B.shape[0], C.shape[0])
    unfolded_tensor = A.dot(khatri_rao(B, C).T)
    return fold(unfolded_tensor, 0, full_shape)

In [38]:
def als(tensor, rank, n_iter_max=200, tol=10e-10):
    
    A = np.random.random_sample((tensor.shape[0], rank))
    B = np.random.random_sample((tensor.shape[1], rank))
    C = np.random.random_sample((tensor.shape[2], rank))

    norm_tensor = l2_norm(tensor)
    error = None

    for iteration in range(n_iter_max):
        # A
        prod = B.T.dot(B)*C.T.dot(C) # TODO
        factor = unfold(tensor, 0).dot(khatri_rao(B, C))
        A = np.linalg.solve(prod.T, factor.T).T

        # B
        prod = A.T.dot(A)*C.T.dot(C)
        factor = unfold(tensor, 1).dot(khatri_rao(A, C))
        B = np.linalg.solve(prod.T, factor.T).T
        
        # C
        prod = A.T.dot(A)*B.T.dot(B)
        factor = unfold(tensor, 2).dot(khatri_rao(A, B))
        C = np.linalg.solve(prod.T, factor.T).T
        
        prev_error = error
        rec = factors_to_tensor(A, B, C)
        error = l2_norm(tensor - rec) / norm_tensor

        if iteration > 1:
            if tol and abs(prev_error - error) < tol:
                print('converged in {} iterations.'.format(
                      iteration))
                break

    return A, B, C


Please generate random tensor. Decompose and reconstruct it with you als method